In [2]:
import cv2
import numpy as np
from keras import layers, models, regularizers, optimizers
import os
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import random
from tqdm import tqdm
from sample_submit.processing import features

In [3]:
def rotation(img, angle):
    angle = int(random.uniform(-angle, angle))
    h, w = img.shape[:2]
    M = cv2.getRotationMatrix2D((int(w/2), int(h/2)), angle, 1)
    img = cv2.warpAffine(img, M, (w, h))
    return img


def brightness(img, low = 0.8, high = 1.2):
    value = random.uniform(low, high)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    hsv = np.array(hsv, dtype = np.float64)
    hsv[:,:,1] = hsv[:,:,1]*value
    hsv[:,:,1][hsv[:,:,1]>255]  = 255
    hsv[:,:,2] = hsv[:,:,2]*value 
    hsv[:,:,2][hsv[:,:,2]>255]  = 255
    hsv = np.array(hsv, dtype = np.uint8)
    img = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    return img

def vertical_shift(img, ratio=0.1):
    if ratio > 1 or ratio < 0:
        print('Value should be less than 1 and greater than 0')
        return img
    ratio = random.uniform(-ratio, ratio)
    h, w = img.shape[:2]
    to_shift = h*ratio
    if ratio > 0:
        img = img[:int(h-to_shift), :, :]
    if ratio < 0:
        img = img[int(-1*to_shift):, :, :]
    img = cv2.resize(img, (h, w), cv2.INTER_CUBIC)
    return img

def channel_shift(img, value):
    value = int(random.uniform(-value, value))
    img = img + value
    img[:,:,:][img[:,:,:]>255]  = 255
    img[:,:,:][img[:,:,:]<0]  = 0
    img = img.astype(np.uint8)
    return img

def augment_image(img, angle = 10, brightness_low = 0.8, brightness_high = 1.2, shift_ratio = 0.1, channel_shift_value = 20):
    img1 = rotation(img, angle)
    img2 = brightness(img, brightness_low, brightness_high)
    img3 = vertical_shift(img, shift_ratio)
    img4 = channel_shift(img, channel_shift_value)
    return (img1, img2, img3, img4)
def grayim(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return gray

In [4]:
train_path = 'prc_train'
i = 0
label_map = {}
allimgs = os.listdir(train_path)
allimgs.sort()

data = []
out = []

for label in tqdm(allimgs):
    label_map[str(i)] = label
    for img in os.listdir(os.path.join(train_path, label)):
        imgmat = cv2.imread(os.path.join(train_path, label, img))
        imgfeat = features(imgmat)
        # print(imgfeat.shape)
        data.append(imgfeat.flatten())
        out.append(i)
    i+=1



100%|██████████| 24/24 [01:45<00:00,  4.40s/it]


In [ ]:
train_path = 'bin_train'
i = 0
label_map = {}
allimgs = os.listdir(train_path)
allimgs.sort()

for label in tqdm(allimgs):
    label_map[str(i)] = label
    for img in os.listdir(os.path.join(train_path, label)):
        imgpath = os.path.join(train_path, label, img)
        imgmat = cv2.imread(imgpath)
        imgfeat = features(imgmat)
        cv2.imwrite(imgpath, imgfeat)

In [5]:
# split into train and test
X_train, X_test, y_train, y_test = train_test_split(np.array(data), np.array(out), test_size=0.15, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((4845, 4096), (855, 4096), (4845,), (855,))

In [16]:
# train_data = []
# train_label = []
# for i,img in enumerate(X_train):
#     train_data.append(grayim(img).flatten())
#     train_label.append(y_train[i])
#     for aug in augment_image(img):
#         train_data.append(grayim(aug).flatten())
#         train_label.append(y_train[i])
# # shuffle
# train_data = np.array(train_data)
# train_label = np.array(train_label)
# p = np.random.permutation(len(train_data))
# train_data = train_data[p]
# train_label = train_label[p]

# # flatten test images
# test_data = []
# test_label = []
# for i,img in enumerate(X_test):
#     test_data.append(grayim(img).flatten())
#     test_label.append(y_test[i])

# X_train = train_data
# y_train = train_label
# X_test = np.array(test_data)
# y_test = np.array(test_label)

# X_train.shape, X_test.shape, y_train.shape, y_test.shape

((24225, 8192), (855, 8192), (24225,), (855,))

In [10]:
clf = SVC(C=5, kernel = 'linear')
clf.fit(X_train, y_train)

SVC(C=5, kernel='linear')

In [11]:
n =0
yp = clf.predict(X_test[n].reshape(1,-1))
yp, y_test[n]

(array([5]), 5)

In [12]:
accuracy_score(y_train, clf.predict(X_train))

1.0

In [13]:
accuracy_score(y_test, clf.predict(X_test))

1.0

In [14]:
import pickle
with open('linear_model_binimg.pkl', 'wb') as f:
    pickle.dump(clf,f)

# import json
# json.dump(label_map, open('label_map.json', 'w') )

In [22]:
labels = open('labels.txt').read().split('\n')
newlabels = []
for i,img in enumerate(os.listdir('sample_submit/test')):
    os.rename(os.path.join('sample_submit/test', img), f'sample_submit/test/{i}.png')
    num = int(img.split('.')[0])
    newlabels.append(labels[num])
with open('sample_submit/code.txt', 'w') as f:
    f.write('\n'.join(newlabels))


In [23]:
from sample_submit.processing import *

img = cv2.imread('train/0.png')
linesremove = remove_lines(img)
cv2.imwrite('linesremoved.png', linesremove)
seg = get_segmented(linesremove)    

for i,s in enumerate(seg):
    cv2.imwrite(f'seg{i}.png', s)

TypeError: remove_lines() missing 1 required positional argument: 'bg'